In [22]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [30]:
from core.utils import ChatChain
from langchain.memory import ChatMessageHistory
import pickle
from datetime import datetime

for _ in range(5):

    interviewer_prompt = open('../agent/prompts/interviewer_prompt.txt', 'r').read()
    interviewer_chat_history = ChatMessageHistory()
    interviewer_agent = ChatChain(interviewer_prompt, temperature=0).create()

    config = {
        'name': 'John Doe',
        'age': 35,
        'gender': 'male',
        'language': 'english'
    }

    auto_patient_prompt = open('../agent/prompts/auto_patient_prompt.txt', 'r').read()
    auto_patient_chat_history = ChatMessageHistory()
    auto_patient_agent = ChatChain(auto_patient_prompt, temperature=0.9).create()

    reply_pairs = []
    print("==========START==========")
    while True:
        interviewer_response = interviewer_agent.invoke(
            {
                **config,
                'chat_history': interviewer_chat_history.messages
            }
        )
        if interviewer_response.endswith('<END OF CONSULTATION>'):
            print("Metis:", interviewer_response.replace('<END OF CONSULTATION>', ''))
            break

        print("Metis:", interviewer_response)
        interviewer_chat_history.add_ai_message(interviewer_response)
        auto_patient_chat_history.add_user_message(interviewer_response)

        patient_response = auto_patient_agent.invoke(
            {
                'chat_history': auto_patient_chat_history.messages
            }
        )
        print("Patient:", patient_response)
        interviewer_chat_history.add_user_message(patient_response)
        auto_patient_chat_history.add_ai_message(patient_response)

        print("=====")
        reply_pairs.append((interviewer_response, patient_response))

    for i, p in reply_pairs:
        

    datetime_str = datetime_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    with open(f'../data/autogen_consultations/consultation_{datetime_str}.pkl', 'wb') as f:
        pickle.dump(reply_pairs, f)

    print("===========END===========")
    print("")

==========START==========
Metis: Hello John! I'm Metis, an AI doctor developed by Platform24. How can I assist you today?
Patient: Chest pain.
=====
Metis: I'm sorry to hear that, John. Can you please describe your chest pain to me? When did it start, and how would you describe the pain - is it sharp, dull, burning, or pressure-like?
Patient: Started this morning. It's a sharp pain.
=====
Metis: I see, John. Does the sharp pain in your chest come and go, or is it constant? And does anything make the pain better or worse?
Patient: It's constant. Nothing makes it better or worse. Also feeling dizzy.
=====
Metis: I'm sorry to hear about the dizziness as well, John. Can you tell me if the chest pain radiates to any other part of your body, like your arms, back, jaw, or neck?
Patient: No, just in my chest.
=====
Metis: Thank you for sharing that, John. Have you experienced any shortness of breath along with the chest pain and dizziness?
Patient: Yes, a bit of shortness of breath too.
=====
